In [1]:
!pip install sqlalchemy psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Load database credentials from environment variables
db_user = os.getenv("PG_USER", "postgres")
db_pass = os.getenv("PG_PASSWORD", "isba_4715")
db_host = os.getenv("PG_HOST", "isba-dev-02.c8p8820o4mvz.us-east-1.rds.amazonaws.com")
db_port = os.getenv("PG_PORT", "5432")
db_name = os.getenv("PG_DB", "sql_project")

# Create the database connection
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')


### Descriptive Analytics
**Business Question:** What are the top 5 highest ranked Apple stores in NYC?

In [16]:
descriptive_query = '''
SELECT "Store Name", "City", CAST("Ranking" AS INTEGER) AS Ranking, "Review"
FROM apple_store_curbed
ORDER BY CAST("Ranking" AS INTEGER) ASC
LIMIT 5;
'''
descriptive_df = pd.read_sql(descriptive_query, engine)
pd.set_option('display.max_rows', None)
descriptive_df

,Store Name,City,ranking,Review
0,Apple Fifth Avenue,New York City,1,Apple Fifth Avenue has become something bigger...
1,Apple Williamsburg,Williamsburg,2,"Bedford Avenue, despite being one of the most ..."
2,Apple Grand Central,New York City,3,All aboard! We’ve made it to the most random A...
3,Apple Upper East Side,New York City,4,Housed in a Beaux Arts building that was once ...
4,Apple Upper West Side,New York City,5,"Another crime-scene Apple Store! In 2021, an N..."


**Insight:** The top-ranked store is Apple Fifth Avenue. The top 5 ranked Apple Stores are not exclusively in Manhattan—only 2 out of 5 are. Surprisingly, Queens, Staten Island, and Brooklyn also host top-ranked stores, despite being considered outer boroughs.

**Recommendation:** Leverage marketing efforts and resources on flagship stores. Reevaluate the performance potential of outer-borough stores.

**Prediction:** Prime locations will continue to lead rankings.

### Diagnostic Analytics
**Business Question:** How do average scores differ between Manhattan vs. outer borough Apple Stores?

In [27]:
diagnostic_query = '''
WITH area_map AS (
  SELECT *,
    CASE
      WHEN "City" IN ('Baychester', 'Elmhurst', 'Heartland Village', 'Downtown Brooklyn') THEN 'Outer NYC'
      ELSE 'NYC'
    END AS Area
  FROM apple_store_curbed
),
ranked_stores AS (
  SELECT Area, "Store Name", "Ranking",
         AVG(CAST("Ranking" AS INTEGER)) OVER (PARTITION BY Area) AS avg_rank_by_area
  FROM area_map
)
SELECT *
FROM ranked_stores
ORDER BY CAST("Ranking" AS INTEGER) ASC;
'''

In [28]:
diagnostic_df = pd.read_sql(diagnostic_query, engine)
pd.set_option('display.max_rows', None)
diagnostic_df

,area,Store Name,Ranking,avg_rank_by_area
0,NYC,Apple Fifth Avenue,1,4.625
1,NYC,Apple Williamsburg,2,4.625
2,NYC,Apple Grand Central,3,4.625
3,NYC,Apple Upper East Side,4,4.625
4,NYC,Apple Upper West Side,5,4.625
5,NYC,Apple West 14th Street,6,4.625
6,NYC,Apple World Trade Center,7,4.625
7,Outer NYC,Apple the Mall at Bay Plaza,8,10.250
8,NYC,Apple Soho,9,4.625
9,Outer NYC,Apple Queens Center,10,10.250


**Insight:** Apple Stores located in Outer NYC areas (e.g., Queens, Staten Island, Brooklyn) have an average ranking of 10.25, significantly worse than the central NYC stores, which consistently average 4.625.

**Recommendation:** Improve experiences or reconsider locations in outer NYC.

**Prediction:** Ranking gap may widen without changes.